# Improve our Agent "4th edition"

## - Install the Required Packages -

In [1]:
%pip install sentence-transformers -q

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\HUAWI\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [2]:
!pip install langchain -q
!pip install transformers -q
!pip install pinecone-client -q
!pip install faiss-cpu -q
!pip install SpeechRecognition -q
!pip install pyttsx3 -q
!pip install -q openai
!pip install -q -U langchain-community

In [3]:
%pip install -q openai==0.28
%pip install -q gTTS
%pip install -q langdetect

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\HUAWI\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\HUAWI\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\HUAWI\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [4]:
%pip install -q ipywebrtc
%pip install -q pvrecorder
%pip install -q sounddevice
%pip install -q gtts
%pip install -q openai


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\HUAWI\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\HUAWI\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\HUAWI\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\HUAWI\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\HUAWI\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


##  - set your OpenAI API key- 

In [5]:
# set OpenAI and chroma API key .env
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

## - Load and Prepare JSON Data -


In [6]:
import json

with open("RimalAI_dataset_expanded.json", "r", encoding="utf-8") as f:
    data = json.load(f)

docs = []
metadatas = []
ids = []

for entry in data:
    # Concatenate relevant fields for embedding
    doc_text = f"{entry['name']} ({entry['type']}): {entry.get('description', '')} Vision 2030: {entry.get('vision2030', '')}"
    docs.append(doc_text)
    metadatas.append({"id": entry["id"], "type": entry["type"], "name": entry["name"]})
    ids.append(str(entry["id"]))


## - Create Embeddings and FAISS Vector Store -

In [7]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

# Use a sentence-transformers model for embeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Create the FAISS vector store
vectordb = FAISS.from_texts(
    texts=docs,
    embedding=embeddings,
    metadatas=metadatas
)

# Save the FAISS index for later use
vectordb.save_local("faiss_rimalai_db")
print("FAISS vector DB created and saved!")


C:\Users\HUAWI\AppData\Local\Temp\ipykernel_7156\3435374507.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


FAISS vector DB created and saved!


## - Query the Vector Database - 

In [8]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

# Custom model
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


# Load the database
vectordb = FAISS.load_local("faiss_rimalai_db", embeddings, allow_dangerous_deserialization=True)

# query
query= "ancient Saudi cities"
results = vectordb.similarity_search(query, k=3)



# Print results
print("Results:")
for doc in results:
    print("Content:", doc.page_content)
    print("Metadata:", doc.metadata)
    print("---")

Results:
Content: Al-Ula (landmark): Al-Ula is an ancient city located in northwestern Saudi Arabia, famous for its sandstone mountains, historic tombs, and rich Nabatean heritage. It has been a crossroads for ancient civilizations and a center of trade and culture. The city is home to significant archaeological sites like Mada'in Saleh, and its unique rock formations make it a prime location for tourists and historians alike. Vision 2030: Al-Ula is a centerpiece of Saudi Arabia's Vision 2030, aiming to transform the city into a world-class tourism destination while preserving its archaeological and cultural heritage. The city is also committed to sustainable tourism practices, ensuring that its natural beauty and historical value are maintained for future generations.
Metadata: {'id': 1, 'type': 'landmark', 'name': 'Al-Ula'}
---
Content: Role of Tribes in Unifying Saudi Arabia (history): The role of various tribes and key figures in the unification of Saudi Arabia, focusing on their c

## LangChain RetrievalQA ("gpt-4o") + Arabic Response

In [8]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

# Initialize the GPT-4 model
llm = ChatOpenAI(model_name="gpt-4o")

# Create the QA chain with retrieval
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectordb.as_retriever()
)

# English query
response = qa_chain("Tell me about Vision 2030 projects in Saudi Arabia.")
print("English Response:", response["result"])

# Arabic query
response = qa_chain("أخبرني عن مشاريع رؤية 2030 في السعودية.")
print("Arabic Response:", response["result"])

C:\Users\HUAWI\AppData\Local\Temp\ipykernel_7156\3646324446.py:5: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-4o")
C:\Users\HUAWI\AppData\Local\Temp\ipykernel_7156\3646324446.py:15: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain("Tell me about Vision 2030 projects in Saudi Arabia.")


English Response: Vision 2030 is Saudi Arabia's ambitious blueprint for economic diversification and modernization, aiming to reduce the country's dependence on oil and develop various sectors such as tourism, culture, and infrastructure. Key projects under Vision 2030 include:

1. **Diriyah Gate Project**: This initiative focuses on restoring and developing Diriyah, the historic birthplace of the Saudi state, into a premier cultural and tourist destination. The project highlights Diriyah's mud-brick architecture and its significance as a UNESCO World Heritage site.

2. **Al-Ula Development**: Al-Ula is being transformed into a world-class tourism destination under Vision 2030. The city, known for its rich Nabatean heritage and unique rock formations, aims to preserve its archaeological and cultural heritage while promoting sustainable tourism practices.

These projects are part of a broader strategy to position Saudi Arabia as a global cultural and economic hub, attracting visitors an

## LangChain RetrievalQA (gpt-4) + Arabic Response

In [11]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

# Initialize the GPT-4 model
llm = ChatOpenAI(model_name="gpt-4")

# Create the QA chain with retrieval
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectordb.as_retriever()
)

# English query
response_english = qa_chain("Tell me about Vision 2030 projects in Saudi Arabia.")
print("English Response:", response_english["result"])

# Arabic query
response_arabic = qa_chain("أخبرني عن مشاريع رؤية 2030 في السعودية.")
print("Arabic Response:", response_arabic["result"])

English Response: Vision 2030 is a strategic framework for the future of Saudi Arabia, with a focus on diversifying the economy, promoting cultural tourism, and creating sustainable urban spaces. Here are some of the key projects under Vision 2030:

1. The Diriyah Gate project: This is a cultural initiative aiming to restore and develop Diriyah, the historic birthplace of the Saudi state. The project seeks to leverage the area's UNESCO World Heritage status and its unique mud-brick architecture to establish Diriyah as a leading cultural and tourist destination.

2. Al-Ula: Located in northwestern Saudi Arabia, Al-Ula is known for its rich Nabatean heritage and significant archaeological sites, including Mada'in Saleh. Under Vision 2030, there are plans to transform Al-Ula into a world-class tourism destination while preserving its archaeological and cultural heritage. The city is also committed to sustainable tourism practices.

3. Neom: A planned city in northwestern Saudi Arabia, Neo

## LangChain RetrievalQA (gpt-3.5-turbo) + Arabic Response

In [12]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

# Initialize the GPT-3.5-turbo model
llm = ChatOpenAI(model_name="gpt-3.5-turbo")

# Create the QA chain with retrieval
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectordb.as_retriever()
)

# English query
response_english = qa_chain("Tell me about Vision 2030 projects in Saudi Arabia.")
print("English Response:", response_english["result"])

# Arabic query
response_arabic = qa_chain("أخبرني عن مشاريع رؤية 2030 في السعودية.")
print("Arabic Response:", response_arabic["result"])

English Response: Saudi Arabia's Vision 2030 includes several key projects aimed at transforming the country in various aspects. Some of these projects include:

1. Diriyah Gate Project: This initiative focuses on restoring and developing Diriyah as a premier cultural and tourist destination, highlighting its historic significance and UNESCO World Heritage status.

2. Al-Ula Transformation: Al-Ula aims to become a world-class tourism destination while preserving its archaeological and cultural heritage. The city is committed to sustainable tourism practices to maintain its natural beauty and historical value.

3. Neom: Neom is a planned city designed to be a hub for technological innovation, sustainable living, and tourism. This project aims to diversify the economy, create sustainable urban spaces, and position Saudi Arabia as a leader in innovation and technology.

Additionally, Vision 2030 promotes Saudi coffee (Gahwa) as part of the country's intangible cultural heritage to enhance

In [9]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

# Initialize the GPT-4 model
llm = ChatOpenAI(model_name="gpt-4o")

# Create the QA chain with retrieval
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectordb.as_retriever()
)

# English query
response_english = qa_chain("Tell me about Al-Ula and give me a video link.")
print("English Response:", response_english["result"])

# Arabic query
response_arabic = qa_chain("أخبرني عن العلا وأعطني رابط فيديو.")
print("Arabic Response:", response_arabic["result"])

English Response: Al-Ula is an ancient city located in northwestern Saudi Arabia, known for its stunning sandstone mountains, historic tombs, and rich Nabatean heritage. It has been a significant crossroads for ancient civilizations and a hub of trade and culture. Al-Ula is home to important archaeological sites like Mada'in Saleh and features unique rock formations that attract tourists and historians alike. As part of Saudi Arabia's Vision 2030, Al-Ula is being developed into a world-class tourism destination, with a focus on preserving its archaeological and cultural heritage while promoting sustainable tourism practices.

I'm unable to provide a video link, but you can find videos about Al-Ula by searching on popular platforms like YouTube for documentaries or travel guides about the region.
Arabic Response: العلا هي مدينة قديمة تقع في شمال غرب المملكة العربية السعودية، وتشتهر بجبالها الرملية وتاريخها الغني بالتراث النبطي. كانت العلا مفترق طرق للحضارات القديمة ومركزاً للتجارة والثق

### Why did the model say "Unfortunately, I can't provide a video link"?

LLM Behavior:
The language model generates answers based on the text context it receives from your vector database. It does not automatically extract or highlight multimedia URLs unless the prompt or chain instructs it to.

Data in Vector Store:
Your dataset does contain video URLs for Al-Ula (e.g., "https://www.youtube.com/watch?v=u7cXaKYLyvs&t=315s"), but the vector store stores text embeddings of concatenated fields. The LLM sees the video URL as part of the text only if it was included in the chunk fed to it.

# fixed code with links to media provided in the prompt

In [14]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
import re

# Set up LLM and RetrievalQA
llm = ChatOpenAI(model_name="gpt-4o")
prompt = PromptTemplate(
    template="""
You are a helpful assistant. Use the context below to answer the question.

Context:
{context}

Question:
{question}
""",
    input_variables=["context", "question"]
)
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectordb.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

# Helper function to detect language
def detect_language(text):
    # Simple heuristic: checks if the text contains Arabic characters
    if re.search(r'[\u0600-\u06FF]', text):
        return 'arabic'
    else:
        return 'english'

# Main query function: Show only the top entity's media links
def query_with_auto_entity(query):
    # Detect the language of the query
    language = detect_language(query)

    docs = vectordb.similarity_search(query, k=5)
    entity_doc = docs[0] if docs else None
    if not entity_doc:
        return "No relevant entry found."
    
    # Get the response from the model (supports Arabic as well)
    response = qa_chain.invoke(query)["result"]
    
    # Retrieve media links from the entity document
    media = entity_doc.metadata.get("media", {})
    video_link = media.get("videos")[0] if media.get("videos") else None
    image_link = media.get("images")[0] if media.get("images") else None
    audio_link = media.get("audio")[0] if media.get("audio") else None
    
    # Prepare the response
    if language == 'arabic':
        # Arabic response
        result = response + "\n\nهذه الروابط ذات الصلة:\n"
        if video_link:
            result += f"- فيديو: [شاهد على يوتيوب]({video_link})\n"
        if image_link:
            result += f"- صورة: ![صورة]({image_link})\n"
        if audio_link:
            result += f"- صوت: [استمع هنا]({audio_link})\n"
        return f"Arabic Query Result:\n{result}"
    
    else:
        # English response
        result = response + "\n\nHere are the relevant media links:\n"
        if video_link:
            result += f"- Video: [Watch on YouTube]({video_link})\n"
        if image_link:
            result += f"- Image: ![Image]({image_link})\n"
        if audio_link:
            result += f"- Audio: [Listen here]({audio_link})\n"
        return f"English Query Result:\n{result}"

# Example usage with two queries: one in English, one in Arabic

# English query
query_english = "Tell me about Al-Ula"
answer_english = query_with_auto_entity(query_english)
print(answer_english)

# Arabic query
query_arabic = "أخبرني عن العلا"
answer_arabic = query_with_auto_entity(query_arabic)
print("\n")
print(answer_arabic)

English Query Result:
Al-Ula is an ancient city located in northwestern Saudi Arabia, renowned for its stunning sandstone mountains, historic tombs, and rich Nabatean heritage. It has historically served as a crossroads for various ancient civilizations and has been a hub of trade and culture. One of its most significant archaeological sites is Mada'in Saleh, which, along with the city's unique rock formations, makes Al-Ula a prime location for tourists and historians.

As part of Saudi Arabia's Vision 2030, Al-Ula is being transformed into a world-class tourism destination. The initiative aims to promote sustainable tourism practices that preserve the city's archaeological and cultural heritage while enhancing its natural beauty and historical significance for future generations.

Here are the relevant media links:



Arabic Query Result:
العلا هي مدينة تاريخية تقع في شمال غرب المملكة العربية السعودية، وتشتهر بجبالها الرملية ومقابرها التاريخية وتراثها النبطي الغني. كانت العلا ملتقى لل

In [15]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import langdetect

# 1. Set up the LLM and RetrievalQA chain
llm = ChatOpenAI(model_name="gpt-4o")

prompt = PromptTemplate(
    template="""
You are a helpful assistant. Use the context below to answer the question.

Context:
{context}

Question:
{question}
""",
    input_variables=["context", "question"]
)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectordb.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

# 2. Helper function to get the first media link by type
def get_first_media_link(media_dict, media_type):
    return media_dict.get(media_type, [None])[0]

# 3. Detect if the query is in Arabic
def detect_language(text):
    try:
        lang = langdetect.detect(text)
        return 'arabic' if lang == 'ar' else 'english'
    except:
        return 'english'

# 4. Main query function: Retrieves answer and media
def query_with_auto_entity(query):
    docs = vectordb.similarity_search(query, k=5)
    entity_doc = docs[0] if docs else None
    if not entity_doc:
        return "No relevant entry found."
    
    response = qa_chain.invoke(query)["result"].strip()

    media = entity_doc.metadata.get("media", {})
    video_link = get_first_media_link(media, "videos")
    image_link = get_first_media_link(media, "images")
    audio_link = get_first_media_link(media, "audio")

    language = detect_language(query)

    if language == 'arabic':
        result = f"{response}\n\nإليك الروابط ذات الصلة:\n"
        if video_link:
            result += f"- فيديو: [شاهد على يوتيوب]({video_link})\n"
        if image_link:
            result += f"- صورة: ![صورة]({image_link})\n"
        if audio_link:
            result += f"- صوت: [استمع هنا]({audio_link})\n"
    else:
        result = f"{response}\n\nHere are the relevant links:\n"
        if video_link:
            result += f"- Video: [Watch on YouTube]({video_link})\n"
        if image_link:
            result += f"- Image: ![Image]({image_link})\n"
        if audio_link:
            result += f"- Audio: [Listen here]({audio_link})\n"

    return result

# 5. Example usage
if __name__ == "__main__":
    queries = [
        "What is Saudi Coffee (Gahwa)?",
        "ما هي القهوة السعودية؟"
    ]
    for query in queries:
        print(f"\nQuery: {query}\n")
        print(query_with_auto_entity(query))
        print("="*80)



Query: What is Saudi Coffee (Gahwa)?

Saudi Coffee (Gahwa) is a traditional beverage in Saudi Arabia that symbolizes hospitality. It is often served with dates during social gatherings and is an important part of the country's cultural heritage. Under Saudi Arabia's Vision 2030, Saudi coffee is promoted as part of the intangible cultural heritage to enhance cultural tourism.

Here are the relevant links:


Query: ما هي القهوة السعودية؟

القهوة السعودية هي نوع من القهوة التقليدية التي تشتهر بها المملكة العربية السعودية. تُحضر القهوة السعودية باستخدام حبوب القهوة الخضراء التي تُحمص بشكل خفيف، ثم تُطحن وتغلى مع الماء. تُضاف إليها مجموعة من التوابل مثل الهيل وأحيانًا الزعفران أو القرنفل لإضفاء نكهة مميزة. تُقدم القهوة السعودية عادة في فناجين صغيرة وتُصاحبها التمور كجزء من الضيافة التقليدية في المملكة. تعتبر القهوة السعودية جزءًا من الثقافة والتراث السعودي، ولها مكانة خاصة في المناسبات الاجتماعية والتجمعات العائلية.

إليك الروابط ذات الصلة:



## 1. Speech-to-Text Using OpenAI Whisper API

In [16]:
import openai
import os
from dotenv import load_dotenv

# Load environment variables from the .env file
load_dotenv()

# Function to transcribe audio using OpenAI's Whisper model
def transcribe_audio(file_path: str) -> str:
    # Open the audio file in binary mode
    with open(file_path, "rb") as audio_file:
        # Call the Whisper model to transcribe the audio
        transcript = openai.Audio.transcribe(
            model="whisper-1",
            file=audio_file
        )
    # Return the transcribed text
    return transcript['text']


In [17]:
# Usage English Audio
text = transcribe_audio("BedouinCulture.mp3")
print("Transcribed text:", text)

Transcribed text: Verbal poetry is the most popular art form in Bedouin culture. It was used not only as an art form, but also as a way to pass down stories through the generations, to convey information, and also to maintain social order in Bedouin society. Poets in Bedouin society were highly respected, and Bedouin poems included accounts of historical events, advice to children of the family, and accounts of battles. Traditionally, poems were recited around the campfire at night, and shared along with other stories. In ancient times, Bedouin couldn't read or write, so they relied on this type of verbal expression to pass on their traditions from one generation to the next.


In [ ]:
# Usage Arabic Audio
text = transcribe_audio("ArabicAudio.m4a")
print("Transcribed text:", text)

Transcribed text: حكم سيوفك في رقاب العُذَّلي وإذا نزلت بدار ظلٍ فرحلي وإذا بليت بظالمٍ كن ظالمًا وإذا لقيت ذو الجهالة فجهلي وإذا الجبان نهاك يومك كريهةٍ خوفاً عليك من ازدحام الجحفلي فاعصي مقالته ولا تحفل بها واقدم إذا حق اللقى في الأول واختر لنفسك منزلاً تعلو به أو متكريماً تحت ظل القصطلي فالموت لا ينجيك من آفاته حصنٌ ولو شيدته بالجندلي موت الفتى في عزةٍ خير له من أن يبي تأسير طرفٍ أكحلي إن كنت في عدد العبيد فهمتي فوق الثريا والسماك الأعزلي أو أنكرت فرسان عبس النسبتي فسنان رمحي والحسام يقر لي وبذابلي ومهندي نلت العلا لا بالقرابة والعديد الأجزلي ورميت مهري في العجاج فخاضه والنار تقدح من شفار الأنصلي يا نازلين على الحما ودياره هل لا رأيتم في الديار تقلقلي طال عزكم وذلي في الهوى ومن العجاء بعزكم وتذللي لا تسقني ماء الحياة بذلة بل فاسقني بالعز كأس الحنظلي ماء الحياة بذلة كجهنم وجهنم بالعز أطيب منزلي لدعمنا قم بتسجيل الاعجاب بالفيديو والاشتراك في القناة مع مشاركة الفيديو


## 2. Text-to-Speech (TTS) Using ElevenLabs Voices
(Expirement Faild))

In [ ]:
import requests
from langdetect import detect
import os

# ElevenLabs API key
ELEVEN_API_KEY = 

# Arabic voice ID from ElevenLabs
VOICE_ID_AR = 
# English voice ID from ElevenLabs
VOICE_ID_EN = 

def text_to_speech_elevenlabs(text, output_file, lang="en"):
    """
    Converts text to speech using ElevenLabs API and saves it as an audio file.
    
    Parameters:
    - text: The text to convert to speech.
    - output_file: The file name to save the output audio.
    - lang: The language of the text ('ar' for Arabic, 'en' for English).
    """
    try:
        # Select URL based on language (Arabic or English)
        if lang == "ar":
            url = f"https://api.elevenlabs.io/v1/text-to-speech/{VOICE_ID_AR}"
        else:
            url = f"https://api.elevenlabs.io/v1/text-to-speech/{VOICE_ID_EN}"

        data = {
            "text": text,
            "model_id": "eleven_monolingual_v1",  # Use "eleven_multilingual_v1" for multilingual support
            "voice_settings": {
                "stability": 0.75,
                "similarity_boost": 0.75
            }
        }

        headers = {
            "xi-api-key": ELEVEN_API_KEY,
            "Content-Type": "application/json"
        }

        # Send the request to the API
        response = requests.post(url, headers=headers, json=data)

        # Save the resulting audio file
        if response.status_code == 200:
            with open(output_file, "wb") as f:
                f.write(response.content)
            print(f"Audio saved successfully as {output_file}")
        else:
            print(f"Request failed: {response.status_code} - {response.text}")
    except Exception as e:
        print(f"An error occurred during text-to-speech conversion: {e}")

def text_to_speech_elevenlabs_pipeline(text, output_file):
    """
    Detects the language of the text and converts it to speech accordingly.
    
    Parameters:
    - text: The text to be converted to speech.
    - output_file: The file name to save the output audio.
    """
    try:
        # Detect language ('en', 'ar', etc.)
        lang = detect(text)
    except Exception:
        lang = 'en'  # fallback to English if language detection fails

    print(f"Detected language: {lang}")
    
    # Convert text to speech using ElevenLabs
    text_to_speech_elevenlabs(text, output_file, lang)

def voice_query_pipeline(audio_file_path: str):
    """
    Handles a voice query: transcribes user speech, retrieves an answer, and converts it to speech.
    
    Parameters:
    - audio_file_path: Path to the input audio file containing the user's voice query.
    
    Returns:
    - output_audio: The path to the output audio file with the answer.
    """
    try:
        # 1. Transcribe user speech to text
        print("Transcribing user speech to text...")
        query_text = transcribe_audio(audio_file_path)
        print("User asked:", query_text)

        # 2. Get answer from LangChain RetrievalQA
        print("Fetching answer from LangChain RetrievalQA...")
        answer_text = query_with_auto_entity(query_text)
        print("Answer:", answer_text)

        # 3. Convert answer text to speech (using ElevenLabs for Arabic/English)
        output_audio = "answer.mp3"
        text_to_speech_elevenlabs_pipeline(answer_text, output_audio)

        # 4. Return or play output_audio as needed
        return output_audio

    except Exception as e:
        print(f"An error occurred during the voice query pipeline: {e}")
        return None

# Example usage
text_en = "Welcome to Al-Ula, a city of ancient wonders and modern vision."
text_ar = "مرحبًا بكم في العلا، مدينة العجائب القديمة والرؤية الحديثة."

# Convert text to speech
text_to_speech_elevenlabs_pipeline(text_en, "alula_en_elevenlabs.mp3")
text_to_speech_elevenlabs_pipeline(text_ar, "alula_ar_elevenlabs.mp3")

# Usage: Assuming the audio file path contains a valid user's question
audio_path = "user_question.wav"
output = voice_query_pipeline(audio_path)

if output:
    print(f"Audio output saved as {output}")
else:
    print("There was an error processing the query.")


Detected language: en
Request failed: 401 - {"detail":{"status":"quota_exceeded","message":"This request exceeds your quota of 10000. You have 0 credits remaining, while 63 credits are required for this request."}}
Detected language: ar
Request failed: 401 - {"detail":{"status":"quota_exceeded","message":"This request exceeds your quota of 10000. You have 0 credits remaining, while 59 credits are required for this request."}}
Transcribing user speech to text...
An error occurred during the voice query pipeline: [Errno 2] No such file or directory: 'user_question.wav'
There was an error processing the query.


In [21]:
def voice_query_pipeline(audio_file_path: str):
    # 1. Transcribe user speech to text
    query_text = transcribe_audio(audio_file_path)
    print("User asked:", query_text)

    # 2. Get answer from LangChain RetrievalQA
    answer_text = query_with_auto_entity(query_text)
    print("Answer:", answer_text)

    # 3. Convert answer text to speech using ElevenLabs (for Arabic/English)
    output_audio = "answer.mp3"
    text_to_speech_elevenlabs_pipeline(answer_text, output_audio)

    # 4. Return or play output_audio as needed
    return output_audio

In [ ]:
# Usage
audio_path = "agent/user_question.wav"
voice_query_pipeline(audio_path)

User asked: هل يمكنك أن تخبرني بشيء أكثر عن كخوة؟
Answer: بالطبع! القهوة السعودية، أو كما تُعرف بـ "القهوة العربية" أو "القهَوة"، هي جزء مهم من الثقافة السعودية وتعتبر رمزًا للضيافة. تُقدم القهوة السعودية تقليديًا مع التمر في المناسبات الاجتماعية والزيارات العائلية والاحتفالات. تتميز القهوة السعودية بنكهتها الفريدة التي تُكتسب من خلال إضافة الهيل وأحيانًا الزعفران أو القرنفل أو الزنجبيل، مما يمنحها طعمًا مميزًا ومختلفًا عن أنواع القهوة الأخرى.

في إطار رؤية المملكة العربية السعودية 2030، يتم الترويج للقهوة السعودية كجزء من التراث الثقافي غير المادي بهدف تعزيز السياحة الثقافية. يُعتبر تقديم القهوة السعودية للضيوف تقليدًا عريقًا يعكس الكرم وحسن الضيافة في المجتمع السعودي.

إليك الروابط ذات الصلة:

Detected language: ar
Request failed: 401 - {"detail":{"status":"quota_exceeded","message":"This request exceeds your quota of 10000. You have 0 credits remaining, while 646 credits are required for this request."}}


'answer.mp3'

In [ ]:
'''# Usage
audio_path = "D:\Project\Q2.WAV"
voice_query_pipeline(audio_path)'''

User asked: اخبرني عن العلم
Answer: يبدو أنك تسأل عن العلم في سياق رؤية 2030 أو تاريخ توحيد السعودية. إذا كنت تستفسر عن العلم في سياق رؤية 2030، فإنه يعكس جهود المملكة العربية السعودية في تعزيز التعليم والبحث العلمي كجزء من خطتها للتحول الوطني. تهدف رؤية 2030 إلى تحسين جودة التعليم وزيادة الاستثمار في البحوث العلمية والتطوير، وذلك لدعم الاقتصاد القائم على المعرفة.

أما إذا كنت تشير إلى العلم في سياق تاريخ توحيد المملكة، فإن التعليم قبل التوحيد كان محدودًا بسبب الظروف القاسية وقلة الموارد. ومع ذلك، بعد توحيد المملكة تحت قيادة الملك عبدالعزيز، بدأت الجهود لتحسين التعليم ونشر المعرفة كجزء من عملية بناء الدولة وتطويرها.

إليك الروابط ذات الصلة:

Detected language: ar
Audio saved successfully as answer.mp3


'answer.mp3'

## add recoding botton 

In [24]:
from pvrecorder import PvRecorder
import wave
import struct

def record_audio(output_file="user_question.wav", duration=5):
    recorder = PvRecorder(frame_length=512)
    recorder.start()
    frames = []
    
    # Record for 'duration' seconds
    for _ in range(int(duration * 16000 / 512)):  # 16000Hz / 512 samples-per-frame
        pcm_frames = recorder.read()
        # Convert 16-bit integer samples to bytes
        bytes_frames = [struct.pack('h', frame) for frame in pcm_frames]
        frames.extend(bytes_frames)
    
    recorder.stop()
    with wave.open(output_file, 'wb') as f:
        f.setnchannels(1)
        f.setsampwidth(2)  # 16-bit = 2 bytes
        f.setframerate(16000)
        f.writeframes(b''.join(frames))
    recorder.delete()

# Test recording
record_audio(duration=25)


## EXPERIMENT 1

In [26]:
from pvrecorder import PvRecorder
import wave
import struct
import openai
from langdetect import detect
import requests
from IPython.display import Audio 

# ElevenLabs API key and voice IDs
ELEVEN_API_KEY = "sk_f039df0fc613d4a45ef33b47ba30cac0dfa0c024ae2e64d1"
VOICE_ID_AR = "IK7YYZcSpmlkjKrQxbSn"
VOICE_ID_EN = "g6xIsTj2HwM6VR4iXFCw"

# Function to convert text to speech using ElevenLabs API
def text_to_speech_elevenlabs(text, output_file, lang="en"):
    try:
        if lang == "ar":
            url = f"https://api.elevenlabs.io/v1/text-to-speech/{VOICE_ID_AR}"
        else:
            url = f"https://api.elevenlabs.io/v1/text-to-speech/{VOICE_ID_EN}"

        data = {
            "text": text,
            "model_id": "eleven_monolingual_v1",
            "voice_settings": {
                "stability": 0.75,
                "similarity_boost": 0.75
            }
        }

        headers = {
            "xi-api-key": ELEVEN_API_KEY,
            "Content-Type": "application/json"
        }

        response = requests.post(url, headers=headers, json=data)

        if response.status_code == 200:
            with open(output_file, "wb") as f:
                f.write(response.content)
            print(f"Audio saved successfully as {output_file}")
        else:
            print(f"Request failed: {response.status_code} - {response.text}")
    except Exception as e:
        print(f"An error occurred during text-to-speech conversion: {e}")

def record_audio(output_file="user_question.wav", duration=5):
    recorder = PvRecorder(frame_length=512)
    recorder.start()
    frames = []
    
    for _ in range(int(duration * 16000 / 512)):  
        pcm_frames = recorder.read()
        bytes_frames = [struct.pack('h', frame) for frame in pcm_frames]
        frames.extend(bytes_frames)
    
    recorder.stop()
    with wave.open(output_file, 'wb') as f:
        f.setnchannels(1)
        f.setsampwidth(2)
        f.setframerate(16000)
        f.writeframes(b''.join(frames))
    recorder.delete()

def voice_query_pipeline():
    # 1. Record audio
    record_audio() 
    
    # 2. Transcribe using Whisper
    audio_file = open("user_question.wav", "rb")
    transcript = openai.Audio.transcribe("whisper-1", audio_file)
    print("You asked:", transcript['text'])
    
    # 3. Get LangChain answer
    answer = query_with_auto_entity(transcript['text'])
    print("Answer:", answer)
    
    # 4. Convert answer to speech using ElevenLabs
    output_audio = "answer.mp3"
    lang = detect(answer)  # Detect the language of the answer
    text_to_speech_elevenlabs(answer, output_audio, lang)
    
    # 5. Return audio player
    return Audio(output_audio)




In [27]:
# Run the full pipeline
voice_query_pipeline()

You asked: هل يمكنك أن تخبرني بمزيد عن الكهوة؟
Answer: بالطبع! الكهوة السعودية، أو القهوة العربية، هي مشروب تقليدي يرمز إلى الضيافة في الثقافة السعودية. تُقدم الكهوة غالبًا مع التمر خلال التجمعات الاجتماعية، وتُعتبر جزءًا من التراث الثقافي غير المادي في السعودية. تتميز الكهوة بطريقة تحضيرها الفريدة التي تشمل استخدام حبوب البن الخضراء والهيل، مما يمنحها نكهة مميزة. تحت رؤية 2030، يتم الترويج للكهوة السعودية كجزء من الجهود لتعزيز السياحة الثقافية، حيث تُعتبر رمزًا للضيافة والتقاليد السعودية الأصيلة.

إليك الروابط ذات الصلة:

Request failed: 401 - {"detail":{"status":"quota_exceeded","message":"This request exceeds your quota of 10000. You have 0 credits remaining, while 473 credits are required for this request."}}


ValueError: rate must be specified when data is a numpy array or list of audio samples.

## EXPERIMENT 2

In [ ]:
from gtts import gTTS
from IPython.display import Audio
import openai


def voice_query_pipeline():
    # 1. Record audio
    record_audio() 
    
    # 2. Transcribe using Whisper
    audio_file = open("user_question.wav", "rb")
    transcript = openai.Audio.transcribe("whisper-1", audio_file)
    print("You asked:", transcript['text'])
    
    # 3. Get LangChain answer
    answer = query_with_auto_entity(transcript['text'])
    print("Answer:", answer)
    
    # 4. Convert to speech
    tts = gTTS(answer)
    tts.save("answer.mp3")
    return Audio("answer.mp3")

# Run the full pipeline
voice_query_pipeline()



NameError: name 'record_audio' is not defined

## EXPERIMENT 3

# Text-To-Speech (TTS) By Google gTTs

In [27]:
from gtts import gTTS
from langdetect import detect

def text_to_speech_gtts(text, output_file):
    # Detect language ('en', 'ar', etc.)
    try:
        lang = detect(text)
    except Exception:
        lang = 'en'  # fallback to English

    # gTTS supports 'en' for English and 'ar' for Arabic
    tts = gTTS(text=text, lang=lang)
    tts.save(output_file)
    print(f"Saved TTS audio to {output_file} (language: {lang})")

# Example usage:
text_en = "Welcome to Al-Ula, a city of ancient wonders and modern vision."
text_ar = "مرحبًا بكم في العلا، مدينة العجائب القديمة والرؤية الحديثة."

text_to_speech_gtts(text_en, "alula_en_gtts.mp3")
text_to_speech_gtts(text_ar, "alula_ar_gtts.mp3")


Saved TTS audio to alula_en_gtts.mp3 (language: en)
Saved TTS audio to alula_ar_gtts.mp3 (language: ar)


In [28]:
def voice_query_from_file(audio_file_path: str):
    # 1. Transcribe user speech to text
    query_text = transcribe_audio(audio_file_path)
    print("User asked:", query_text)

    # 2. Get answer from LangChain RetrievalQA
    answer_text = query_with_auto_entity(query_text)
    print("Answer:", answer_text)

    # 3. Convert answer text to speech
    output_audio = "answer.mp3"
    text_to_speech_gtts(answer_text, output_audio)

    # 4. Return or play output_audio as needed
    return output_audio


In [29]:
# Usage
audio_path = r"D:\Project\Q1.WAV"
voice_query_from_file(audio_path)

User asked: Tell me about King Abdulaziz.
Answer: King Abdulaziz Al Saud, also known as Ibn Saud, played a crucial role in the formation of modern Saudi Arabia. He is credited with unifying the Arabian Peninsula, transforming it from a collection of fragmented tribes and rival territories into the cohesive Kingdom of Saudi Arabia. His leadership was marked by a combination of military strategy, diplomatic efforts, and a profound understanding of the region's cultural dynamics. King Abdulaziz's efforts to unite the tribes were vital in establishing the kingdom and laying the foundations for its future development. Under his rule, the country began to transition from a series of tribal societies into a unified nation-state.

Here are the relevant links:

Saved TTS audio to answer.mp3 (language: en)


'answer.mp3'

In [30]:
# Usage
audio_path = r"D:\Project\Q2.WAV"
voice_query_from_file(audio_path)

User asked: اخبرني عن العلم
Answer: يبدو أن سؤالك يتعلق بالعلم، لكن السياق المتاح يركز على تاريخ وتوحيد المملكة العربية السعودية ورؤية 2030. إذا كنت تسأل عن العلم في سياق رؤية 2030، فإن رؤية المملكة العربية السعودية 2030 تتضمن خططًا لتطوير التعليم والبحث العلمي كجزء من جهودها لتنويع الاقتصاد وتحسين جودة الحياة في المملكة. هل ترغب في معرفة المزيد عن جانب محدد من رؤية 2030 أو عن موضوع آخر متعلق بالعلم؟

إليك الروابط ذات الصلة:

Saved TTS audio to answer.mp3 (language: ar)


'answer.mp3'

voice/text input, detects language, and responds in text/voice using the dataset and requirements

In [1]:
%pip install speechrecognition pyaudio langid gtts openai

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\HUAWI\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [34]:
from gtts import gTTS
from IPython.display import Audio, display
import openai
import speech_recognition as sr
from langdetect import detect


def record_audio(duration=5, filename="user_question.wav"):
    """Record audio with automatic language detection"""
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print("Listening...")
        audio = r.record(source, duration=duration)
        with open(filename, "wb") as f:
            f.write(audio.get_wav_data())
        return filename

def detect_language(text):
    """Detect language using langdetect"""
    try:
        lang = detect(text)
        return lang
    except:
        return 'en'  # fallback
    
def voice_query_interactive():
    # Input selection
    input_method = input("Enter 'v' for voice or 't' for text: ").lower()
    
    # 1. Input capture
    if input_method == 'v':
        audio_file = record_audio()
        with open(audio_file, "rb") as f:
            transcript = openai.Audio.transcribe("whisper-1", f)
        user_text = transcript['text']
    else:
        user_text = input("Enter your question: ")
    
    print("\nYou asked:", user_text)
    
    # 2. Language detection
    lang = detect_language(user_text)
    print(f"Detected language: {lang}")
    
    # 3. Get LangChain answer
    answer = query_with_auto_entity(user_text)
    print("\nAnswer:", answer)
    
    # 4. Convert to speech
    tts = gTTS(answer, lang=lang if lang in ['en', 'fr', 'es','ar'] else 'en')  # Add more supported languages as needed
    tts.save("answer.mp3")
    
    # 5. Output methods
    output_method = input("\nEnter 'v' for voice or 't' for text output: ").lower()
    
    if output_method == 'v':
        return Audio("answer.mp3")
    else:
        return answer

# Run the full pipeline
result = voice_query_interactive()
if isinstance(result, Audio):
    display(result)
else:
    print("\nText Answer:", result)


Listening...

You asked: Tell me about New York.
Detected language: en

Answer: New York is a state in the northeastern United States and is one of the most populous and influential states in the country. Its capital is Albany, but its largest and most famous city is New York City, often referred to simply as "New York."

### New York City:
- **Boroughs**: The city is composed of five boroughs: Manhattan, Brooklyn, Queens, The Bronx, and Staten Island.
- **Economy**: It is a major global economic hub, home to Wall Street and the New York Stock Exchange, making it a critical center for finance and business.
- **Culture and Arts**: New York City is renowned for its cultural diversity, vibrant arts scene, and iconic landmarks such as Times Square, Central Park, the Statue of Liberty, and Broadway.
- **Education and Innovation**: The city hosts numerous prestigious universities and institutions, including Columbia University and New York University, and is a leader in innovation and techno

In [43]:
# Run the full pipeline
result = voice_query_interactive()
if isinstance(result, Audio):
    display(result)
else:
    print("\nText Answer:", result)

Listening...

You asked: Tell me about 2030 vision.
Detected language: en

Answer: Vision 2030 is a strategic framework aimed at reducing Saudi Arabia's dependence on oil, diversifying its economy, and developing public service sectors such as health, education, infrastructure, recreation, and tourism. Under this vision, several key cultural and tourism projects have been initiated to transform Saudi Arabia into a global tourism hub while preserving its rich cultural heritage.

For instance, the Diriyah Gate project is a significant cultural initiative under Vision 2030, focused on restoring and developing Diriyah—known for its historic mud-brick architecture and UNESCO World Heritage status—into a premier cultural and tourist destination.

Similarly, Al-Ula, with its ancient sandstone mountains and historic Nabatean sites like Mada'in Saleh, is being transformed into a world-class tourism destination. The initiative emphasizes sustainable tourism practices to preserve the city's archa

# - Model Evaluation -

#### A- Retrieval Evaluation

In [ ]:
def evaluate_and_diagnose(query, expected_ids, vectordb, k=3):
    # Step 1: Retrieve top-k documents
    results = vectordb.similarity_search(query, k=k)
    retrieved_ids = [doc.metadata.get("id", None) for doc in results]

    # Step 2: Match IDs
    found_ids = [doc_id for doc_id in expected_ids if doc_id in retrieved_ids]
    
    # Step 3: Compute Metrics
    precision = len(found_ids) / k
    recall = len(found_ids) / len(expected_ids) if expected_ids else 0
    mrr = sum([1 / (retrieved_ids.index(doc_id) + 1) for doc_id in found_ids]) / len(found_ids) if found_ids else 0

    # Step 4: Print Results
    print(f"\n🔍 Query: {query}")
    print(f"📊 Precision@{k}: {precision:.2f}")
    print(f"📍 Recall@{k}: {recall:.2f}")
    print(f"📈 MRR: {mrr:.2f}")
    print("\n📄 Retrieved Documents:")
    for i, doc in enumerate(results):
        print(f"  [{i+1}] ID: {doc.metadata.get('id')} — {doc.page_content[:200].strip()}...\n")

    # Step 5: Diagnose missing expected_ids
    missing_ids = [doc_id for doc_id in expected_ids if doc_id not in retrieved_ids]
    if missing_ids:
        print(f"⚠️ Missing expected IDs: {missing_ids}")
    else:
        print("✅ All expected IDs retrieved!")

    # Step 6: Auto Diagnosis
    if not found_ids:
        print("🧠 Diagnosis: No relevant docs retrieved.")
        print("🔍 Possible issues:")
        print("- ❌ Query not semantically similar to content.")
        print("- ❌ Embedding model is weak or not tuned.")
        print("- ❌ Documents may not contain relevant terms.")
    elif precision < 0.5:
        print("🧠 Diagnosis: Low precision.")
        print("🔧 Consider improving embeddings or increasing k.")
    elif recall < 0.5:
        print("🧠 Diagnosis: Low recall.")
        print("📂 May need better matching logic or more relevant content.")
    else:
        print("✅ Retrieval is working reasonably well.")

    return precision, recall, mrr

In [3]:
# Test cases
test_cases = [
    ("Al-Ula", [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]),
    ("NEOM", [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]),
    ("Bedouin Culture", [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]),
    ("Diriyah", [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]),
    ("King Abdulaziz", [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]),
    ("Pre-Unification Saudi Arabia", [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]),
    ("How Oil Transformed Saudi Arabia", [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]),
]


In [4]:
for query, expected in test_cases:
    result = evaluate_and_diagnose(query, expected, vectordb)
    print(f"Query: '{query}' | Expected ID: {expected} | Found: {result}")

NameError: name 'vectordb' is not defined

In [68]:
for query, expected_id in test_cases:
    result = test_retrieval(query, expected_id)
    print(f"Query: '{query}' | Expected ID: {expected_id} | Found: {result}")

Query: 'Al-Ula' | Expected ID: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12] | Found: (0.25, 0.25, 0.611111111111111)
Query: 'NEOM' | Expected ID: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12] | Found: (0.25, 0.25, 0.611111111111111)
Query: 'Bedouin Culture' | Expected ID: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12] | Found: (0.25, 0.25, 0.611111111111111)
Query: 'Diriyah' | Expected ID: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12] | Found: (0.25, 0.25, 0.611111111111111)
Query: 'King Abdulaziz' | Expected ID: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12] | Found: (0.25, 0.25, 0.611111111111111)
Query: 'Pre-Unification Saudi Arabia' | Expected ID: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12] | Found: (0.25, 0.25, 0.611111111111111)
Query: 'How Oil Transformed Saudi Arabia' | Expected ID: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12] | Found: (0.25, 0.25, 0.611111111111111)


#### B- QA Response Quality

In [70]:
pip install -q rouge-score


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\HUAWI\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [71]:
from rouge_score import rouge_scorer

# Def Function to Evaluate Response Quality
def evaluate_response(query, reference_answer):
    generated = qa_chain.invoke(query)["result"]
    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    scores = scorer.score(reference_answer, generated)
    return scores


In [ ]:
# Query
query = "ما هي أهمية العلا في التاريخ؟"
reference = "العلا كانت مركزًا حضاريًا وتجاريًا هامًا للأنباط في شمال غرب الجزيرة العربية."

evaluate_response(query, reference)


{'rougeL': Score(precision=0, recall=0, fmeasure=0)}

In [74]:
generated = qa_chain.invoke(query)["result"]
print(f"Generated: {generated}")
print(f"Reference: {reference}")

Generated: العلا لها أهمية كبيرة في التاريخ لعدة أسباب. تقع في شمال غرب المملكة العربية السعودية وكانت مركزًا للحضارات القديمة ومفترق طرق للتجارة والثقافة. تعتبر العلا موطنًا لمواقع أثرية مهمة مثل مدائن صالح، وهي معروفة بتشكيلاتها الصخرية الفريدة والجبال الرملية. يعود تراثها الغني إلى الحضارة النبطية، مما يجعلها وجهة رئيسية للسياح والمؤرخين. كما أنها تلعب دورًا محوريًا في رؤية السعودية 2030، حيث تهدف المملكة إلى تحويل العلا إلى وجهة سياحية عالمية مع الحفاظ على تراثها الأثري والثقافي.
Reference: العلا كانت مركزًا حضاريًا وتجاريًا هامًا للأنباط في شمال غرب الجزيرة العربية.


In [ ]:
# Query
query2 = "What is the importance of Al-Ula in history?"
reference2 = "Al-Ula was an important cultural and commercial center for the Nabataeans in the northwest of the Arabian Peninsula."

evaluate_response(query2, reference2)


{'rougeL': Score(precision=0.09523809523809523, recall=0.42105263157894735, fmeasure=0.1553398058252427)}

In [78]:
generated2 = qa_chain.invoke(query2)["result"]
print(f"Generated: {generated2}")
print(f"Reference: {reference2}")

Generated: Al-Ula is an ancient city located in northwestern Saudi Arabia that holds significant historical importance due to its role as a crossroads for ancient civilizations and a center of trade and culture. The city is renowned for its sandstone mountains, historic tombs, and rich Nabatean heritage. Al-Ula is home to significant archaeological sites like Mada'in Saleh, which highlight its historical value and attract tourists and historians alike. Its unique rock formations and well-preserved cultural heritage make it a crucial site for understanding the history and development of the region. Additionally, Al-Ula's historical significance is being preserved and promoted as part of Saudi Arabia's Vision 2030, aiming to transform it into a world-class tourism destination while maintaining its archaeological and cultural heritage for future generations.
Reference: Al-Ula was an important cultural and commercial center for the Nabataeans in the northwest of the Arabian Peninsula.


In [ ]:
# Query No3.
query_diriyah = "What is the importance of Diriyah in history?"
reference_diriyah = "Diriyah is historically significant as the birthplace of the Saudi state. It is known for its distinctive mud-brick architecture and holds UNESCO World Heritage status. Diriyah's importance is tied to its role in the early history of Saudi Arabia, serving as the original seat of power for the Al Saud family. It was a center of political and religious leadership and played a crucial role in the early unification efforts that eventually led to the formation of the modern Kingdom of Saudi Arabia. Under Saudi Arabia's Vision 2030, Diriyah is being restored and developed as a premier cultural and tourist destination, highlighting its historical and cultural significance."


generated_diriyah = qa_chain.invoke(query_diriyah)["result"]
print(f"Generated: {generated_diriyah}")
print(f"Reference: {reference_diriyah}")

Generated: Diriyah holds significant historical importance as it is the birthplace of the Saudi state. It is celebrated for its distinctive mud-brick architecture and has been recognized as a UNESCO World Heritage site. Diriyah played a crucial role in the early history of Saudi Arabia, serving as a center for the initial establishment and expansion of the Saudi state under the leadership of the Al Saud family. Its restoration and development into a premier cultural and tourist destination is a key initiative under Saudi Arabia's Vision 2030, reflecting its enduring cultural and historical significance.
Reference: Diriyah is historically significant as the birthplace of the Saudi state. It is known for its distinctive mud-brick architecture and holds UNESCO World Heritage status. Diriyah's importance is tied to its role in the early history of Saudi Arabia, serving as the original seat of power for the Al Saud family. It was a center of political and religious leadership and played a c

In [82]:
evaluate_response(query_diriyah, reference_diriyah)

{'rougeL': Score(precision=0.5151515151515151, recall=0.4636363636363636, fmeasure=0.48803827751196166)}

In [ ]:
# Query
query_jinn = "What are the beliefs and practices surrounding jinn in Saudi culture?"
reference_jinn = """
Jinn are supernatural beings deeply embedded in both Islamic theology and Saudi folklore. While they are mentioned in the Qur'an as intelligent beings created from smokeless fire, in Saudi Arabia, cultural stories and practices surrounding jinn extend into desert myths, traditional warnings, and spiritual beliefs. Saudis often believe in different categories of jinn, including neutral jinn like Al-Jan, evil jinn like Shayateen, and powerful jinn like Ifrit and Marid. They inhabit places like caves, deserts, old wells, ruins, and uninhabited homes. To avoid disturbing them, Saudis commonly say 'Bismillah' before entering unfamiliar places.
Common themes in folklore include jinn appearing as animals, guiding or misleading travelers with lights, and whispers heard in deserted places. Spiritual practices, such as reciting Ayat al-Kursi and seeking protection from evil jinn through prayers like Surat Al-Falaq and Surat An-Nas, are used to protect oneself. Despite their modernized portrayal in films and media, the belief in jinn remains strong, particularly in rural areas.
"""
generated_jinn = qa_chain.invoke(query_jinn)["result"]
print(f"Generated: {generated_jinn}")
print(f"Reference: {reference_jinn}")


Generated: In Saudi culture, beliefs and practices surrounding jinn are deeply intertwined with both Islamic theology and local folklore. Jinn are considered supernatural beings created from smokeless fire, as mentioned in the Qur'an. They are intelligent and possess free will, similar to humans. In Saudi folklore, jinn are often featured in desert myths and traditional stories, where they are sometimes portrayed as tricksters or beings that can influence human affairs.

Cultural practices may include various traditional warnings and spiritual beliefs related to jinn. For instance, people might take precautions to avoid disturbing jinn, such as not throwing hot water on the ground without saying a protective prayer, as it is believed that this might harm or anger them. Jinn are also thought to inhabit certain areas, like deserted or unclean places, and people may avoid these locations at night.

Overall, jinn hold a significant place in the cultural and spiritual landscape of Saudi Ara

In [84]:
evaluate_response(query_jinn, reference_jinn)


{'rougeL': Score(precision=0.42424242424242425, recall=0.25766871165644173, fmeasure=0.3206106870229008)}

#### C- Multilingual Testing

In [85]:
# حالة الاختبار الأولى
arabic_test_cases = [
    ("الجن في الثقافة السعودية", "الجن مخلوقات خارقة تم ذكرها في القرآن الكريم، خلقها الله من نارٍ بلا دخان. يُعتقد في الثقافة السعودية أن الجن يعيشون في أماكن مهجورة أو غير نظيفة، مثل الصحاري والمقابر.")
]
# حالة الاختبار الثانية
arabic_test_cases.append(
    ("أهمية الجن في التراث السعودي", "الجن جزء من الموروث الثقافي والديني في السعودية. يعتقد السعوديون أن الجن يختلفون في نوعياتهم، مثل الجن الطيبين (الجان) والشياطين، والجن الأقوياء مثل العفاريت. يُعتقد أن الجن قد يظهرون في شكل حيوانات أو يرشدون المسافرين.")
)

# حالة الاختبار الثالثة
arabic_test_cases.append(
    ("معلومات عن الدرعية", "الدرعية تعدّ مهد الدولة السعودية، وقد اشتهرت بموقعها الفريد وعمارتها الطينية. تم إدراجها في قائمة التراث العالمي لليونسكو، وهي تحتفظ بأهمية تاريخية كبيرة كونها كانت مركزًا لتأسيس الدولة السعودية تحت قيادة آل سعود.")
)

# حالة الاختبار الرابعة
arabic_test_cases.append(
    ("الدرعية ورؤية السعودية 2030", "الدرعية تعدّ جزءًا هامًا من رؤية السعودية 2030، حيث يتم تطويرها لتصبح وجهة سياحية وثقافية رئيسية تعكس تاريخ المملكة وإرثها الثقافي.")
)


In [86]:
# الدالة لاختبار النموذج على حالات الاختبار
def run_bilingual_tests():
    for query, ref_answer in arabic_test_cases:
        # استعلام النموذج للحصول على الإجابة
        response = qa_chain.invoke(query)["result"]
        
        # طباعة الاستعلام والنتيجة المرجعية والنتيجة المولدة
        print(f"Query: {query}")
        print(f"Response: {response}")
        print(f"Reference: {ref_answer}")
        print("-" * 50)  # فاصل لتوضيح الحدود بين الاختبارات

# تنفيذ الاختبارات
run_bilingual_tests()

Query: الجن في الثقافة السعودية
Response: الجن في الثقافة السعودية هم كائنات خارقة للطبيعة متجذرة بعمق في كل من اللاهوت الإسلامي والفولكلور السعودي. على الرغم من ذكرهم في القرآن الكريم ككائنات ذكية خُلقت من نار بدون دخان، إلا أن القصص والممارسات الثقافية المرتبطة بالجن في السعودية تمتد إلى أساطير الصحراء والتحذيرات التقليدية والمعتقدات الروحية.
Reference: الجن مخلوقات خارقة تم ذكرها في القرآن الكريم، خلقها الله من نارٍ بلا دخان. يُعتقد في الثقافة السعودية أن الجن يعيشون في أماكن مهجورة أو غير نظيفة، مثل الصحاري والمقابر.
--------------------------------------------------
Query: أهمية الجن في التراث السعودي
Response: الجِنّ لها مكانة بارزة في التراث الثقافي السعودي والعربي بشكل عام. في الثقافة التقليدية، يُعتبر الجن مخلوقات غير مرئية تعيش في عالم موازٍ للبشر، ولها قدرات خارقة. تُذكر الجن في العديد من القصص والأساطير التي تُروى عبر الأجيال، وتلعب دوراً في الفولكلور والحكايات الشعبية. 

تظهر قصص الجن في الأدب العربي الكلاسيكي، مثل "ألف ليلة وليلة"، حيث تُستخدم كوسيلة لتفسير الظواهر غير ال

In [88]:
# Define the test cases for English queries
english_test_cases = [
    ("Jinn in Saudi culture", "Jinn in Saudi culture are supernatural beings deeply rooted in both Islamic theology and Saudi folklore. While they are mentioned in the Qur'an as intelligent beings created from smokeless fire, cultural stories and practices surrounding jinn in Saudi Arabia extend into desert myths, traditional warnings, and spiritual beliefs."),
    ("Importance of jinn in Saudi heritage", "Jinn hold a prominent place in Saudi and Arab cultural heritage. In traditional culture, they are believed to be invisible beings that exist in a parallel world to humans and possess supernatural powers. Jinn appear in many stories and legends passed down through generations and play a role in folklore and popular tales."),
    ("Information about Diriyah", "Diriyah is a historic city located in the Riyadh region of Saudi Arabia, and it is the birthplace of the first Saudi state. Established in the mid-15th century, Diriyah became the capital of the first Saudi state in 1744 under the leadership of Imam Muhammad bin Saud. The city is famous for its strategic location on Wadi Hanifah and its traditional mud-brick architecture."),
    ("Diriyah and Saudi Vision 2030", "Diriyah is historically significant as the birthplace of the Saudi state, with its distinctive mudbrick architecture and UNESCO World Heritage status. Under Saudi Vision 2030, the Diriyah Gate project aims to restore and develop Diriyah as a leading cultural and tourist destination, focusing on enhancing Saudi Arabia's cultural heritage while promoting tourism and economic diversification.")
]

# Function to run bilingual tests (for English queries)
def run_bilingual_tests():
    for query, ref_answer in english_test_cases:
        # Query the model to get the response
        response = qa_chain.invoke(query)["result"]
        
        # Print the query, reference answer, and generated response
        print(f"Query: {query}")
        print(f"Response: {response}")
        print(f"Reference: {ref_answer}")
        print("-" * 50)  # Separator to clarify the boundaries between tests

# Run bilingual tests
run_bilingual_tests()

Query: Jinn in Saudi culture
Response: In Saudi culture, jinn are considered supernatural beings that play a significant role in both Islamic theology and local folklore. They are described in the Qur'an as intelligent beings created from smokeless fire. In Saudi Arabia, jinn are deeply embedded in cultural stories, desert myths, traditional warnings, and spiritual beliefs. These narratives often reflect the country's historical and environmental context, such as the harsh desert conditions and the importance of oral traditions. Jinn stories are part of the fabric of Saudi cultural identity, contributing to the rich tapestry of myths and legends in the region.
Reference: Jinn in Saudi culture are supernatural beings deeply rooted in both Islamic theology and Saudi folklore. While they are mentioned in the Qur'an as intelligent beings created from smokeless fire, cultural stories and practices surrounding jinn in Saudi Arabia extend into desert myths, traditional warnings, and spiritual

#### D- End-to-End Pipeline

In [93]:
pip install pydub

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\HUAWI\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [94]:
from pydub import AudioSegment

def convert_to_wav(input_file, output_file):
    audio = AudioSegment.from_file(input_file)
    audio.export(output_file, format="wav")

# استخدام الدالة لتحويل الملف
convert_to_wav(r"D:\Project\EngQ3.WAV", "user_question.wav")


c:\Users\HUAWI\AppData\Local\Programs\Python\Python39\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
c:\Users\HUAWI\AppData\Local\Programs\Python\Python39\lib\site-packages\pydub\utils.py:198: RuntimeWarning: Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work
  warn("Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work", RuntimeWarning)


FileNotFoundError: [WinError 2] The system cannot find the file specified

In [92]:
import time
import speech_recognition as sr
from gtts import gTTS

# Function to transcribe audio to text using speech_recognition library
def transcribe_audio(audio_file):
    recognizer = sr.Recognizer()
    with sr.AudioFile(audio_file) as source:
        audio = recognizer.record(source)
    try:
        # Using Google Speech Recognition to transcribe the audio
        transcript = recognizer.recognize_google(audio, timeout=10)
        return transcript
    except sr.UnknownValueError:
        return "Could not understand the audio"
    except sr.RequestError:
        return "Error in the speech recognition service"
    except TimeoutError:
        return "Request timed out, please check your internet connection"

# Function for querying with automatic entities (can be customized as needed)
def query_with_auto_entity(transcript):
    # This function could extract some information from the transcript
    # NLP techniques can be used here for additional processing
    answer = f"The transcript is: {transcript}"
    return answer

# Function to convert text to speech using gTTS
def text_to_speech_gtts(text, output_file):
    tts = gTTS(text)
    tts.save(output_file)

# Function to test the full pipeline
def full_pipeline_test(audio_file):
    start_time = time.time()  # Start measuring time

    # Convert audio to text
    transcript = transcribe_audio(audio_file)
    print(f"Transcript: {transcript}")

    # Query with automatic entity processing to get an answer
    answer = query_with_auto_entity(transcript)
    print(f"Answer: {answer}")

    # Convert the answer to speech and save it in a file
    output_file = "test_output.mp3"
    text_to_speech_gtts(answer, output_file)
    
    end_time = time.time()  # End measuring time
    elapsed_time = end_time - start_time
    print(f"Full Pipeline Test took {elapsed_time} seconds.")

    return output_file  # Return the name of the output file

# Run the test
audio_file = r"D:\Project\EngQ3.WAV" # Make sure this file exists on your device
output_file = full_pipeline_test(audio_file)

print(f"Output file saved as: {output_file}")


ValueError: Audio file could not be read as PCM WAV, AIFF/AIFF-C, or Native FLAC; check if file is corrupted or in another format

#### E- Test

In [104]:
topics = [
    "Jinn in Saudi culture",
    "Importance of jinn in Saudi heritage",
    "Information about Diriyah",
    "Diriyah and Saudi Vision 2030"
]


In [105]:
import time

def benchmark_query(query, iterations=10):
    latencies = []
    for _ in range(iterations):
        start = time.time()
        response = qa_chain.invoke(query)
        latencies.append(time.time() - start)
        
    avg_latency = sum(latencies) / iterations
    print(f"Average response time: {avg_latency:.2f} seconds")
    
    # Check if the average response time is within the target (less than 2 seconds)
    if avg_latency < 2:
        print("Performance is within acceptable limits.")
    else:
        print("Performance is slower than expected.")
    
    return avg_latency

# Test the benchmark query
query = "What is Saudi Coffee?"
benchmark_query(query)


Average response time: 1.79 seconds
Performance is within acceptable limits.


1.7883833646774292

In [106]:
def availability_test(query, total_requests=100):
    failed_requests = 0
    for _ in range(total_requests):
        try:
            response = qa_chain.invoke(query)
        except Exception as e:
            print(f"Error: {e}")
            failed_requests += 1
    
    failure_rate = failed_requests / total_requests * 100
    print(f"Failure rate: {failure_rate:.2f}%")
    
    # Check if the failure rate is within the acceptable limit (less than 1%)
    if failure_rate < 1:
        print("Service is highly available.")
    else:
        print("Service is experiencing failures.")
        
# Test the availability
availability_test(query)


Failure rate: 0.00%
Service is highly available.


In [107]:
import concurrent.futures

def concurrent_queries(query, num_users=10):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [executor.submit(benchmark_query, query) for _ in range(num_users)]
        results = [future.result() for future in concurrent.futures.as_completed(futures)]
    
    # Check if the system can handle multiple users without significant slowdowns
    avg_latency = sum(results) / len(results)
    print(f"Average response time with {num_users} users: {avg_latency:.2f} seconds")
    
    # Check if performance remains within acceptable limits
    if avg_latency < 2:
        print("System supports multiple users effectively.")
    else:
        print("System performance degrades under load.")

# Test the system with multiple users
concurrent_queries(query)


Average response time: 1.68 seconds
Performance is within acceptable limits.
Average response time: 1.73 seconds
Performance is within acceptable limits.
Average response time: 1.75 seconds
Performance is within acceptable limits.
Average response time: 1.76 seconds
Performance is within acceptable limits.
Average response time: 1.80 seconds
Performance is within acceptable limits.
Average response time: 1.80 seconds
Performance is within acceptable limits.
Average response time: 1.98 seconds
Performance is within acceptable limits.
Average response time: 2.02 seconds
Performance is slower than expected.
Average response time: 2.18 seconds
Performance is slower than expected.
Average response time: 2.33 seconds
Performance is slower than expected.
Average response time with 10 users: 1.90 seconds
System supports multiple users effectively.


#### F- User Simulation Testing

In [108]:
user_scenarios = [
    {
        "user_type": "Tourist",
        "sample_queries": [
            "What are the coordinates of AlUla?",
            "What is the best season to visit AlUla?",
            "What are the Bedouin customs in Saudi Arabia?",
            "Who is Mohammed bin Salman?"
        ],
        "expected_keywords": ["AlUla", "winter", "Bedouin", "Mohammed bin Salman"]
    },
    {
        "user_type": "Student / Learner",
        "sample_queries": [
            "What is Vision 2030?",
            "Explain Bedouin culture in simple terms",
            "What is the difference between NEOM and AlUla?"
        ],
        "expected_keywords": ["Vision 2030", "Bedouin", "NEOM", "AlUla", "culture", "future"]
    },
    {
        "user_type": "Journalist / Content Creator",
        "sample_queries": [
            "What was Saudi Arabia like before oil?",
            "Give a summary of the tribal role in the unification of Saudi Arabia.",
            "How did Bedouins use the stars?"
        ],
        "expected_keywords": ["oil", "tribes", "unification", "Bedouins", "stars", "navigation"]
    }
]


In [113]:
def simulate_user_queries(qa_chain, scenarios):
    results = []

    for scenario in scenarios:
        print(f"\n👤 Testing for user type: {scenario['user_type']}")
        for query in scenario["sample_queries"]:
            response = qa_chain.invoke(query)

            # Extract the text from the response
            response_text = response.get("result", str(response))

            # Check if any expected keywords appear in the response
            matched_keywords = [
                keyword for keyword in scenario["expected_keywords"]
                if keyword.lower() in response_text.lower()
            ]
            success = len(matched_keywords) > 0

            # Store the test result
            results.append({
                "user_type": scenario["user_type"],
                "query": query,
                "response": response_text,
                "matched_keywords": matched_keywords,
                "success": success
            })

            # Print the test outcome
            print(f"🟢 Query: {query}")
            print(f"📝 Response: {response_text}")
            print(f"✅ Match: {matched_keywords if matched_keywords else '❌ No match'}\n")

    return results


In [114]:
results = simulate_user_queries(qa_chain, user_scenarios)



👤 Testing for user type: Tourist
🟢 Query: What are the coordinates of AlUla?
📝 Response: The coordinates of AlUla are approximately 26.7° N latitude and 37.9° E longitude.
✅ Match: ['AlUla']

🟢 Query: What is the best season to visit AlUla?
📝 Response: The best season to visit Al-Ula is during the cooler months, typically from October to April. During this time, the weather is more comfortable for exploring the outdoor archaeological sites, rock formations, and engaging in various outdoor activities. The temperatures are milder, making it an ideal time for tourists to experience the cultural and historical attractions of Al-Ula.
✅ Match: ❌ No match

🟢 Query: What are the Bedouin customs in Saudi Arabia?
📝 Response: Bedouin customs in Saudi Arabia are deeply influenced by their nomadic lifestyle and the harsh desert environment in which they live. Key customs include:

1. **Mobility and Adaptability**: Bedouins traditionally move from place to place in search of water and pasture for t

In [115]:
success_rate = sum([1 for r in results if r["success"]]) / len(results)
print(f"\n✅ Success Rate: {success_rate*100:.2f}%")



✅ Success Rate: 90.00%


#### G- LangSmith Evaluators

In [136]:
! pip install -q langchain langsmith openai

In [137]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [149]:
from langsmith.evaluation import EvaluationResult, run_evaluator
from langdetect import detect

@run_evaluator
def qa_accuracy(run, example) -> EvaluationResult:
    predicted = run.outputs.get("output", run.outputs.get("result", ""))
    reference = example.outputs.get("answer", "")
    score = 1.0 if reference.lower() in predicted.lower() else 0.0
    return EvaluationResult(key="qa_accuracy", score=score)

@run_evaluator
def language_match(run, example) -> EvaluationResult:
    input_text = example.inputs.get("question", "")
    output_text = run.outputs.get("output", run.outputs.get("result", ""))
    try:
        input_lang = detect(input_text)
        output_lang = detect(output_text)
        score = 1.0 if input_lang == output_lang else 0.0
    except:
        score = 0.0
    return EvaluationResult(key="language_match", score=score)

In [ ]:
from langsmith import Client
client = Client()

# Create dataset (only need to run once)
try:
    dataset = client.create_dataset(
        dataset_name="rimalai-qa-test",
        description="Test QA pairs for RimalAI evaluation",
    )
    
    # Add test cases
    client.create_examples(
        dataset_id=dataset.id,
        inputs=[
            {"question": "Tell me about Al-Ula."},
            {"question": "أخبرني عن القهوة السعودية."},
        ],
        outputs=[
            {"answer": "Al-Ula is an ancient city in Saudi Arabia"},
            {"answer": "القهوة السعودية هي مشروب تقليدي"},
        ]
    )
except:
    print("Dataset likely already exists") 

Dataset likely already exists


In [151]:
from langchain.smith import RunEvalConfig

eval_config = RunEvalConfig(
    evaluators=[qa_accuracy, language_match],
    custom_evaluators=[qa_accuracy, language_match],
)

results = client.run_on_dataset(
    dataset_name="rimalai-qa-test",
    llm_or_chain_factory=lambda: qa_chain,  # Your existing QA chain
    evaluation=eval_config,
    verbose=True,
)

View the evaluation results for project 'worthwhile-cod-64' at:
https://smith.langchain.com/o/30b55858-cb4b-4ec8-9d7a-f967bb3caa5c/datasets/e06f2a55-077b-4d33-ba68-db6b5cf83a9b/compare?selectedSessions=adbb80c8-b9c3-417f-87f8-b9d44ee27464

View all tests for Dataset rimalai-qa-test at:
https://smith.langchain.com/o/30b55858-cb4b-4ec8-9d7a-f967bb3caa5c/datasets/e06f2a55-077b-4d33-ba68-db6b5cf83a9b
[------------------------------------------------->] 2/2

,feedback.qa_accuracy,feedback.language_match,error,execution_time,run_id
count,2.0,2.0,0,2.000000,2
unique,NaN,NaN,0,NaN,2
top,NaN,NaN,NaN,NaN,f442820d-76f1-48d7-b1af-089e5b6a324d
freq,NaN,NaN,NaN,NaN,1
mean,0.0,0.0,NaN,2.844160,NaN
std,0.0,0.0,NaN,0.727787,NaN
min,0.0,0.0,NaN,2.329537,NaN
25%,0.0,0.0,NaN,2.586849,NaN
50%,0.0,0.0,NaN,2.844160,NaN
75%,0.0,0.0,NaN,3.101471,NaN


In [152]:
print(f"Evaluation complete. View results at: https://smith.langchain.com/datasets/rimalai-qa-test")

Evaluation complete. View results at: https://smith.langchain.com/datasets/rimalai-qa-test


# tools

### 1. Mapbox

In [59]:
load_dotenv()

MAPBOX_TOKEN = os.getenv("MAPBOX_TOKEN_KEY")
if not MAPBOX_TOKEN:
    raise ValueError("not found in environment variables")

In [60]:
MAPBOX_TOKEN

'pk.eyJ1IjoiYWFsc2hlaHJpcmFuZWVtIiwiYSI6ImNtYTVneGc5ZzBnOXUyanNkNjlhd2c0dGMifQ.18PwQfDB7VSz3lLaI1Yetw'

In [61]:

response = requests.get(f"https://api.mapbox.com/styles/v1/mapbox/streets-v11?access_token={MAPBOX_TOKEN}")
print(response.status_code)  # Should print 200 if the token works

200


In [64]:
import os
import requests
from urllib.parse import quote_plus
from dotenv import load_dotenv
from IPython.display import Image, display

# Load environment variables
load_dotenv()

# Get Mapbox token
MAPBOX_TOKEN = os.getenv("MAPBOX_TOKEN_KEY")
if not MAPBOX_TOKEN:
    raise ValueError("MAPBOX_TOKEN_KEY not found in environment variables")

# Test if token works (should print 200)
test_response = requests.get(f"https://api.mapbox.com/styles/v1/mapbox/streets-v11?access_token={MAPBOX_TOKEN}")
print("Token Test Status Code:", test_response.status_code)  # Should print 200

# === Functions ===
def locate_place(search_query):
    """Forward geocode place name to coordinates using Mapbox API v5 (restricted to Saudi Arabia)"""
    base_url = "https://api.mapbox.com/geocoding/v5/mapbox.places/"
    query_encoded = quote_plus(search_query)
    url = f"{base_url}{query_encoded}.json"
    
    # Bounding box for Saudi Arabia: [minLon, minLat, maxLon, maxLat]
    saudi_bbox = [34.0, 16.0, 56.0, 32.0]  # Rough box around Saudi borders
    
    params = {
        "access_token": MAPBOX_TOKEN,
        "limit": 1,
        "types": "poi,place,address",
        "bbox": ",".join(map(str, saudi_bbox))  # restrict search to Saudi Arabia
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        features = data.get("features", [])
        if features:
            feature = features[0]
            coords = feature["geometry"]["coordinates"]  # [lng, lat]
            place_name = feature.get("place_name", "Unknown place")
            return coords, place_name
    return None, None

def get_static_map(lnglat, zoom=14, size=(600, 400)):
    """Generate Mapbox Static Map URL with marker"""
    return (f"https://api.mapbox.com/styles/v1/mapbox/streets-v12/static/"
            f"pin-l-circle+ff0000({lnglat[0]},{lnglat[1]})/"
            f"{lnglat[0]},{lnglat[1]},{zoom}/{size[0]}x{size[1]}"
            f"?access_token={MAPBOX_TOKEN}")

def location_chatbot():
    while True:
        place = input("\nEnter place name (or 'quit' to exit): ")
        if place.lower() == "quit":
            break
        coords, name = locate_place(place)
        if coords:
            print(f"📍 Found: {name}")
            map_url = get_static_map(coords)
            display(Image(url=map_url))
        else:
            print("⚠️ Location not found. Please try again.")

# Run chatbot
location_chatbot()


Token Test Status Code: 200
📍 Found: Addiriyah, Riyadh, Saudi Arabia


📍 Found: Desert Spring Village Residence Street, Al Tanyah First, Dubai, Dubai, United Arab Emirates


📍 Found: طريق الخيل, Al Jadaf, Dubai, Dubai, United Arab Emirates


📍 Found: Al Rub' Al Khali Road, Alkhubar, Eastern, Saudi Arabia


In [44]:
pip install streamlit pydeck pandas

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached protobuf-5.29.4-cp38-abi3-macosx_10_9_universal2.whl.metadata (592 bytes)
Using cached protobuf-5.29.4-cp38-abi3-macosx_10_9_universal2.whl (417 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.0rc1
    Uninstalling protobuf-6.31.0rc1:
      Successfully uninstalled protobuf-6.31.0rc1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.72.0rc1 requires protobuf<7.0dev,>=6.30.0, but you have protobuf 5.29.4 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [75]:
MAPBOX_TOKEN

'pk.eyJ1IjoiYWFsc2hlaHJpcmFuZWVtIiwiYSI6ImNtYTVneGc5ZzBnOXUyanNkNjlhd2c0dGMifQ.18PwQfDB7VSz3lLaI1Yetw'

In [76]:
import pydeck as pdk
import pandas as pd

# ✅ Set token globally (this avoids the TypeError)
pdk.settings.mapbox_api_key = MAPBOX_TOKEN

map_style = f"https://api.mapbox.com/styles/v1/mapbox/satellite-streets-v12?access_token={MAPBOX_TOKEN}"

# Example Saudi landmark
icon_data = pd.DataFrame({
    'lat': [26.6912],
    'lon': [37.9236],
    'name': ['Al-Ula (Hegra)'],
    'icon': ['marker']
})

icon_layer = pdk.Layer(
    'IconLayer',
    icon_data,
    get_icon='icon',
    get_size=4,
    size_scale=15,
    get_position='[lon, lat]',
    pickable=True,
    icon_atlas="https://raw.githubusercontent.com/visgl/deck.gl-data/master/website/icon-atlas.png",
    icon_mapping={
        "marker": {"x": 0, "y": 0, "width": 128, "height": 128, "anchorY": 128}
    }
)

r = pdk.Deck(
    layers=[icon_layer],
    initial_view_state=view_state,
    map_style=map_style,  # ✅ correct HTTP style URL,
    tooltip={"text": "{name}"}
)

r.show()